<a href="https://colab.research.google.com/github/sarthak-314/100-Days-in-Data/blob/master/BERT%20with%20Hugging%20Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**BERT**

In [ ]:
import pandas as pd
import torch
import os
CSV_PATH = './Colab Notebooks/CADABRA/Sentiment Analysis/Amazon/cleaned_reviews.csv'
%cd /content/drive/My\ Drive
#Getting the reviews
reviews = pd.read_csv(CSV_PATH)

/content/drive/My Drive


Install Hugging Face transformers library

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

#Downloading a pretrained wordpiece tokenizer for bert-base-uncased
TOKENIZER = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
#Trying out the tokenizer on sentence from random text generator
rand = 'Carriage quitting securing be appetite. Have to leave happy.'
#Encode the sentence
encoded_random_sentence = TOKENIZER.encode_plus(rand, return_tensors='pt')

print('Random Sentence:', rand)
print('Tokenized sentence:', TOKENIZER.tokenize(rand))
print('Encoded sentence:', encoded_random_sentence)

Random Sentence: Carriage quitting securing be appetite. Have to leave happy.
Tokenized sentence: ['carriage', 'quit', '##ting', 'securing', 'be', 'appetite', '.', 'have', 'to', 'leave', 'happy', '.']
Encoded sentence: {'input_ids': tensor([[  101,  9118,  8046,  3436, 12329,  2022, 18923,  1012,  2031,  2000,
          2681,  3407,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


TOKENIZER.tokenize() splits the sentence into words or more specifically tokens

TOKENIZER.encode_plus() returns -
* input_ids : Indices of input tokens in the vocabulary
* attention_mask: Mask to avoid performing attention on padding token indices
* token_type_ids: Segment token indices to indicate first and second portions of the inputs. This is useful in situations like question-answering. However, in sentiment analyzer all token_type_ids will have the same value of 0


### **DATASET**

In [ ]:
from torch.utils.data import Dataset
MAX_LEN = 512

In [ ]:
class BookReviewDataset(Dataset):
    #Here target is to predict the rating from the review
    def __init__(self, review, target):
        super().__init__()
        self.review = review
        self.target = target
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN

    def __len__(self):
        return len(self.target)
    
    def __getitem__(self, item):
        review = self.review[item]
        inputs = self.tokenizer.encode_plus(review, add_special_tokens=True, max_length=self.max_len, return_tensors='pt', padding='max_length')
        targets = torch.tensor(self.target[item], dtype=torch.float)
        return {
            **inputs,
            'targets': targets
        }

In [ ]:
#Trying out the Dataset
df = reviews.head(3)
x, y = df['text'].values, df['rating'].values
dataset = BookReviewDataset(review=x, target=y)
for i in range(len(dataset)):
    sample = dataset[i]
    review_text = df.text.iloc[i]
    rating = df.rating.iloc[i]
    print('Review text:', review_text)
    print('Rating:', rating)
    print(sample)


Review text: doesn't really give any real practical advice...bunch of general gibberish...
Rating: 1.0
{'input_ids': tensor([[  101,  2987,  1005,  1056,  2428,  2507,  2151,  2613,  6742,  6040,
          1012,  1012,  1012,  9129,  1997,  2236, 21025, 29325,  4509,  1012,
          1012,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0

The input ids contains 512 token ids. It starts with 101 ([SEP] token) and end with 102 ([CLS] token). After the sentence is over, the rest is padded with zeros





### **BERT MODEL**

In [ ]:
import torch.nn as nn
from transformers import AutoModel
#Downloading BERT from Hugging Face 
BERT = AutoModel.from_pretrained('bert-base-uncased')

Let's take a look at bert-base-uncased

In [ ]:
BERT

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

The output features in BERT = 768

Let's try to run bert through the encoded_random_sentence

In [ ]:
last_hidden_state, pooler_output, hidden_states = BERT(**encoded_random_sentence, output_hidden_states=True)
print('last hidden states:', last_hidden_state)
print('Shape:', last_hidden_state.shape)
print('pooler output:', pooler_output)
print('Shape:', pooler_output.shape)
print('hidden states', hidden_states)
print('Shape:', len(hidden_states))

last hidden states: tensor([[[-0.3164,  0.1807,  0.5991,  ..., -0.5167,  0.2474,  0.2024],
         [ 0.6553, -0.0325,  0.6342,  ..., -0.4374, -0.0837,  0.2381],
         [ 0.1808, -0.2434,  0.6021,  ..., -0.3035, -0.4888, -0.6548],
         ...,
         [-0.0572,  0.0222,  1.0353,  ..., -0.4381, -0.2263, -0.5108],
         [-0.3055, -0.7431,  0.4558,  ...,  0.2426,  0.2778, -0.6339],
         [ 0.4754,  0.4112, -0.1788,  ..., -0.1181, -0.5148, -0.1388]]],
       grad_fn=<NativeLayerNormBackward>)
Shape: torch.Size([1, 14, 768])
pooler output: tensor([[-0.8461, -0.3591, -0.8256,  0.8139,  0.5849, -0.1358,  0.8525,  0.3443,
         -0.7474, -1.0000, -0.1433,  0.8811,  0.9740,  0.4459,  0.9196, -0.7437,
         -0.3211, -0.5182,  0.3184, -0.5139,  0.6059,  1.0000,  0.1605,  0.1797,
          0.3644,  0.9821, -0.6937,  0.9101,  0.9493,  0.6321, -0.7054,  0.1186,
         -0.9833, -0.1474, -0.8892, -0.9867,  0.3318, -0.6311,  0.0289,  0.0258,
         -0.8764,  0.1908,  1.0000,  0.1872,

In [ ]:
torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)

tensor([[[-0.1497, -0.1190, -0.1204,  ..., -0.5167,  0.2474,  0.2024],
         [ 1.2780,  0.0215,  0.7973,  ..., -0.4374, -0.0837,  0.2381],
         [ 0.1676,  0.2337,  0.3537,  ..., -0.3035, -0.4888, -0.6548],
         ...,
         [-0.2093,  0.0136,  1.3205,  ..., -0.4381, -0.2263, -0.5108],
         [-0.6572, -0.0281, -0.3348,  ...,  0.2426,  0.2778, -0.6339],
         [ 0.0066, -0.0350,  0.0436,  ..., -0.1181, -0.5148, -0.1388]]],
       grad_fn=<CatBackward>)

In [ ]:
pooled_output = torch.cat([hidden_states[i] for i in [-4, -3, -2, -1]], dim=-1)
pooled_output = pooled_output[:, 0, :]
pooled_output

tensor([[-0.1497, -0.1190, -0.1204,  ..., -0.5167,  0.2474,  0.2024]],
       grad_fn=<SliceBackward>)

Last hidden state outputs a tensor of shape (1, 14, 768) with values between -1 to 1

1 is the batch size, 14 is the input sentence size+2 (for [SEP] and [CLS]) and 768 is the output size for bert



In [ ]:
class BERTBase(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.bert = BERT
        self.dropout = nn.Dropout(0.1)
        self.lin = nn.Linear(3072, 5)
        self.classifier = nn.Softmax(dim=5)

    def forward(self, input_ids, token_type_ids, attention_mask):
        _, _, hidden_states = self.bert(input_ids, attention_mask, token_type_ids, output_hidden_states=True)
        #Huggingface documentation recommends averaging / pooling sequence of hidden states
        #concating last 4 layers
        pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
        pooled_output = pooled_output[:, 0, :]
        # pooled_output = self.dropout(pooled_output)
        lin = self.lin(pooled_output)
        logits = self.classifier(lin)
        return lin

Trying out BERT

In [ ]:
bert = BERTBase()
output = bert(**encoded_random_sentence)
output

tensor([[ 0.1178,  0.3417, -0.1257,  0.1926,  0.6664]],
       grad_fn=<AddmmBackward>)

The model outputs **5** numbers that add to 1. 

These numbers are the *probabilities* the model assigns to the rating of the review.The 5 values are probabilities that the review has - 1 star, 2 star, 3 star, 4 star or 5 star rating.

We select the maximum value (in this case 2 star) as the output of the model. This model has not been trained so the output is random for the sentence

**ENGINE**

In [ ]:
from tqdm import tqdm
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

BATCH_SIZE = 4
TRAIN_BATCH_SIZE = 32
EPOCHS = 4

In [ ]:
def get_loss(outputs, targets):
    loss = nn.CrossEntropyLoss()
    return loss(outputs, targets)

In [ ]:
sentence ='The book was awesome'
encoded = TOKENIZER.encode_plus(sentence, return_tensors='pt')
BERTBase()(**encoded)
# get_loss(output, [[1, 0, 0, 0, 0]])

tensor([[ 0.1688,  0.6081,  0.0689,  0.1867, -0.5106]],
       grad_fn=<AddmmBackward>)

In [ ]:
out, 

tensor([[ 0.3460, -0.4786,  0.3860, -0.0064,  0.2565]],
       grad_fn=<AddmmBackward>)

In [ ]:
get_loss(out, [[1]])

AttributeError: ignored

In [ ]:
def train(data_loader, bert, optimizer):
    bert.train()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for batch, d in tqdm(enumerate(data_loader)): 
        input_ids = d['input_ids'].to(device)
        token_type_ids = d['token_type_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        
        optimizer.zero_grad()
        outputs = bert(input_ids, token_type_ids, attention_mask)
        loss = get_loss(outputs, targets)
        loss.backward()
        optimizer.step()
    

In [ ]:
def evaluate(data_loader, bert):
    bert.eval()
    total_targets, total_outputs = [], []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for bi, d in tqdm(enumerate(data_loader)):
        input_ids = d['input_ids'].to(device)
        token_type_ids = d['token_type_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        targets = d['targets'].to(device)
        total_targets.append(targets.cpu().detach().numpy.to_list())

        outputs = bert(input_ids, token_type_ids, attention_mask)
        total_outputs.append(outputs.cpu().detach().numpy().to_list())
    
    return total_outputs, total_target

Running the code

In [ ]:
df = reviews.head(100)
df_train, df_valid = train_test_split(df)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

train_dataset = BookReviewDataset(review=df_train.text, target=df_train.rating)
valid_dataset = BookReviewDataset(review=df_valid.text, target=df_valid.rating)

train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE)
valid_dataloader = DataLoader(valid_dataset, batch_size = BATCH_SIZE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert = BERTBase()
bert.to(device)

param_optimizer = list(bert.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

optimizer = Adam(optimizer_parameters)
# for epoch in range(EPOCHS):
#     train(train_dataloader, bert, optimizer)
#     outputs, targets = evaluate(valid_dataloader, bert)
#     # outputs = [x>0.5 for x in outputs] or some tshit
#     accuracy = metrics.accuracy_score(outputs, targets)
#     if accuracy > best_accuracy:
#         bert.save(bert.state_dict()) #TODO: Add save path
#         best_accuracy = accuracy



In [ ]:
train(train_dataloader, bert, optimizer)

0it [00:00, ?it/s]


RuntimeError: ignored

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
run()

NameError: ignored

**TRAIN**

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

BATCH_SIZE = 4
#TODO: fuc is num_workers
NUM_WORKERS = 8

In [ ]:
train, valid = train_test_split(X, stratify=X.rating)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
train_dataset = BookReviewDataset(review=train.text, target=train.rating)
valid_dataset = BookReviewDataset(review=valid.text, target=valid.rating)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = BERT()


In [ ]:
from sklearn.model_selection import train_test_split
X, Y = df[['summary', 'text']], df['rating']
X_train, X_valid = train_test_split(X)